In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
import numpy as np
import pandas as pd
import scipy.linalg
import time

from src.graph import visualize
from src.recommender import train_mf, reviews_dataset, evaluate_recommendations
from src.recommender.mf_recommender import train_mf
from src.tags import PytorchWordEmbedding

In [3]:
reviews_train, reviews_validation, reviews_test, images = reviews_dataset()

In [4]:
recommender = train_mf(
    reviews_train,
    images,
    d = 20,
    alpha = .01,
    beta = .01,
    als_epochs = 1,
    sgd_learning_rate = .1,
    sgd_epochs = 10,
    sgd_batch_size = 10,
)

Precomputing reviews_by_user
Elapsed: 46.18s
Computing X
[    0 35968 10499 32248 26487  9514 38288 23690  5346 21356 37936 38944
 13802 20983 52820 10748 22679  7967  5207  7048 71851  4910  3656   174
  8715  2350 83690  8948 41410  2230 33478  7056 84910  2055 54357  5950
 91352]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 4. 5. 4. 3. 5. 4. 5. 5. 5.
 4. 5. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5.]
[     1    745  63310  27030   7343   5938  12248  12248  12248  16451
  34537  17645  24787   6370  20436  26073  58840  41724  38499  12447
  75634  44786 113232  17866   7736  22310  20087   9719  18799  17522
  13181  13181  12179   1681   1681  27358  38370   6598  17571   7480
   4927  20219  15001  83022  99226    108   1106  12775  84360  39823
  19299   3405   3405   5113   1395   7508   7063  58194  21753  52323
   2048  77964   5705  10035  42351  39066  59455  72639   9515  22350
   5447  10728 103098  91462   9004  34646  39386  13324  13324  10090
  10090   3844  19701  17415  2657

RuntimeError: 

In [ ]:
reviews_validation_by_user = reviews_validation.groupby("user_index").apply(lambda group: (group["image_index"].to_numpy(), group["rating"].to_numpy()))

In [ ]:
validation_loss = 1/n * sum(1 / len(image_ids) * sum((ratings - image_tags[image_ids] @ (Y @ X[u])) ** 2) for u, (image_ids, ratings) in reviews_validation_by_user.items() if len(image_ids) > 0)
validation_loss

In [ ]:
recommender = train_knn(reviews_train, word_embedding)

In [ ]:
visualize(recommender.graph, {vertex: vertex.id for vertex in recommender.graph.vertices})

In [ ]:
evaluate_recommendations(recommender, reviews_validation)